# VacationPy

In [15]:
# dependencies and setup
import pandas as pd
import requests
import gmaps
import os

# import API key. Will have to create your own api_keys.py that stores your api key in the variable g_key
from api_keys import g_key

# configuring gmaps
gmaps.configure(api_key=g_key)

### Storing Results into DataFrame

In [16]:
path = pd.read_csv('../output_data/cities.csv', index_col=[0])
cities_df = pd.DataFrame(path)
cities_df.head()

,City,Country,Date,Max Temp,Humidity,Cloudiness,Wind Speed,Lat,Long
0,Tasiilaq,GL,1584217467,5.00,65,20,1.12,65.61,-37.64
1,Auki,SB,1584217467,84.33,79,100,15.88,-8.77,160.70
2,Hervey Bay,AU,1584217468,66.99,82,90,5.82,-25.30,152.85
3,Airai,TL,1584217468,63.99,83,18,2.91,-8.93,125.41
4,Salalah,OM,1584217469,77.00,61,75,8.05,17.02,54.09


### Humidity Heatmap

In [17]:
figure_layout = {
    'width': '1300px',
    'height': '700px',
    'padding': '1px',
    'margin': '0 auto 0 auto'}

In [18]:
# the weight variable will store humidity values upon which the heat_layer will render its results
# consult gmaps documentation
locations = cities_df[['Lat', 'Long']]
weight = cities_df['Humidity'].astype(float)

heat_map = gmaps.figure(layout=figure_layout)
heat_layer = gmaps.heatmap_layer(locations, weights=weight,dissipating=False, max_intensity=250,
                                 point_radius=6)

heat_map.add_layer(heat_layer)

embed_minimal_html('export.html', views=[heat_map])

heat_map

Figure(layout=FigureLayout(height='700px', margin='0 auto 0 auto', padding='1px', width='1300px'))

In [19]:
# creating a new dataframe that contains values that could make for a comfortable vacation:
# 70˚F min, 85˚F max, 15% cloudiness max, 5mph wind speed max

ideal_weather = cities_df.loc[(cities_df['Max Temp'] >= 70) &
                               (cities_df['Max Temp'] <=85) &
                               (cities_df['Cloudiness'] <= 15) &
                               (cities_df['Wind Speed'] <= 5), :]


ideal_weather = ideal_weather.reset_index()
ideal_weather = ideal_weather.drop(['index'],axis=1)

ideal_weather = pd.DataFrame(ideal_weather)

### Hotel Map

In [20]:
# base url for Google Maps API
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

In [21]:
# storing parameters as a dictionary for readability
params = {
    "key": g_key,
    "keyword": 'hotel',
    "radius": 5000,
    "type": 'lodging'}

ideal_weather['Hotel Name'] = ''

# 
for index, row in ideal_weather.iterrows():
    
    lat = row["Lat"]
    long = row["Long"]
    params['location'] = f'{lat},{long}'
    
    hotels = requests.get(base_url, params=params).json()
    
    try:
        ideal_weather.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


In [22]:
ideal_weather.head()

,City,Country,Date,Max Temp,Humidity,Cloudiness,Wind Speed,Lat,Long,Hotel Name
0,Castro,BR,1584217486,73.17,59,0,2.06,-24.79,-50.01,Central Palace Hotel - Castro (PR)
1,Avanigadda,IN,1584217559,73.89,85,0,4.97,16.02,80.92,
2,Kyabé,TD,1584217667,80.73,9,0,4.25,9.45,18.94,
3,Adrar,MR,1584217753,70.47,8,0,3.89,20.50,-10.07,
4,Marawi,PH,1584217821,75.00,85,1,1.01,8.00,124.29,Marawi Resort Hotel


In [23]:
hotel_df = ideal_weather.loc[ideal_weather['Hotel Name'] != '']
hotel_df

,City,Country,Date,Max Temp,Humidity,Cloudiness,Wind Speed,Lat,Long,Hotel Name
0,Castro,BR,1584217486,73.17,59,0,2.06,-24.79,-50.01,Central Palace Hotel - Castro (PR)
4,Marawi,PH,1584217821,75.00,85,1,1.01,8.00,124.29,Marawi Resort Hotel
5,Yola,NG,1584217802,75.69,10,0,0.40,9.20,12.48,City Green Hotel
6,Caldas Novas,BR,1584217744,76.41,85,6,3.31,-17.74,-48.62,diRoma Fiori
7,Tuscaloosa,US,1584217840,80.01,57,1,3.36,33.21,-87.57,Hotel Indigo Tuscaloosa Downtown
8,Bago,MM,1584217915,77.00,83,0,4.70,17.34,96.48,Kanbawza Hinthar
9,Gandai,IN,1584217929,72.81,75,0,3.76,21.67,81.10,"""SHRADDHA MAA KRIPA HOTEL""(SEVAK HOTEL)"
12,Rantauprapat,ID,1584218117,73.89,96,0,2.06,2.10,99.83,Permata Land Hotel & Resort
14,Yanam,IN,1584218202,72.09,89,0,1.86,16.73,82.22,Capital O 62066 G.v.s Residency
16,Maddūr,IN,1584218374,72.81,60,0,2.42,12.60,77.07,S A B Residency


In [24]:
# the info box template was given. Will pop up over arrow symbols added to the heatmap. 
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Long"]]

In [25]:
symbols = gmaps.symbol_layer(
    locations, fill_color='red', stroke_color='red', info_box_content=hotel_info)
heat_map.add_layer(symbols)

heat_map

Figure(layout=FigureLayout(height='700px', margin='0 auto 0 auto', padding='1px', width='1300px'))